In [1]:
import json
import re
from collections import Counter

import pandas as pd

In [2]:
with open('data/daily_dataset_v3.json', 'r', encoding="utf8") as f:
    daily = json.load(f)
    
with open('data/topical_dataset_v3.json', 'r', encoding="utf8") as f:
    topical = json.load(f)

In [11]:
a = "AbC"
a.isupper()

False

## Daily

In [3]:
target_sents = [sample['predict'] for sample in daily]
target_sents = [sample for sample in target_sents if len(sample['entities']) == 1]
for sample in target_sents:
    sample['entities'] = sample['entities'][0]

In [4]:
df = pd.json_normalize(target_sents)

In [5]:
df[df.midas == "yes_no_question"].head()

,text,midas,entities.label,entities.offsets,entities.text
1,are they from LOCATION?,yes_no_question,location,"[14, 20]",taiwan
9,would you like these PRODUCT delivered or woul...,yes_no_question,product,"[21, 29]",supplies
17,"sure, is dry white FOOD ok?",yes_no_question,food,"[19, 23]",wine
51,do you really have all of these PRODUCT?,yes_no_question,product,"[32, 37]",drugs
54,"do you ever see the ANIMAL you trained, like o...",yes_no_question,animal,"[20, 24]",dogs


In [60]:
df[['midas']].value_counts()

midas                
statement                4108
opinion                  1907
yes_no_question          1003
open_question_factual    500 
command                  430 
open_question_opinion    342 
comment                  110 
complaint                78  
neg_answer               67  
dev_command              61  
pos_answer               60  
appreciation             1   
other_answers            1   
dtype: int64

In [61]:
df[['entities.label']].value_counts()

entities.label           
product                      2368
location                     1019
food                         845 
business                     638 
org                          489 
work_of_art                  454 
event                        418 
occupation                   411 
fac                          298 
law                          292 
film                         272 
academic_discipline          269 
language                     194 
person                       158 
literary_work                125 
type_of_sport                106 
norp                         85  
nation                       63  
song                         53  
music_genre                  28  
sports_event                 22  
animal                       16  
chemical_element             13  
sport_team                   8   
sports_season                8   
sports_venue                 7   
political_party              6   
association_football_club    2   
championship          

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

df[df['entities.label'] == 'occupation']['entities.text'].value_counts()
# df[(df['entities.label'] =='work_of_art')]
# df[(df['entities.label'] =='work_of_art') & (df['entities.text'] =='bye')]

C:\Users\denis\AppData\Local\Temp/ipykernel_2628/3418291369.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


doctor                            37
boss                              34
manager                           27
job                               26
position                          22
salary                            18
teacher                           14
president                         12
driver                            12
officer                           9 
guy                               8 
secretary                         6 
employee                          6 
worker                            6 
employer                          5 
leader                            5 
man                               5 
employees                         4 
stewardess                        4 
person                            4 
staff                             4 
care                              4 
supervisor                        4 
agent                             3 
guide                             3 
charge                            3 
inspector                         3 
p

In [43]:
rare_midas = ['other_answers']
rare_entity = ['sport_team', 'sports_season', 'sports_venue','political_party', "association_football_club", "championship"]
df_common_midas = df[~df.midas.isin(rare_midas)]
df_rare_midas = df[df.midas.isin(rare_midas)]
df_common_entity = df[~df['entities.label'].isin(rare_entity)]

In [44]:
common_sampled_by_midas = df_common_midas.groupby('midas', group_keys=False).apply(lambda x: x.sample(10))
common_sampled_by_entity = df_common_entity.groupby('entities.label', group_keys=False).apply(lambda x: x.sample(10))
rare_sampled_by_midas = df_rare_midas.groupby('midas', group_keys=False).apply(lambda x: x.sample(1))

In [45]:
pd.concat([common_sampled_by_midas[['midas', 'text']], 
           rare_sampled_by_midas[['midas', 'text']]]).to_csv('data/daily_midas_sample.tsv', sep='\t', index=False)

In [46]:
common_sampled_by_entity[['entities.label', 'text']].to_csv('data/daily_entity_sample.tsv', sep='\t', index=False)

In [47]:
df['midas_entity'] = df[['midas', 'entities.label']].agg(' '.join, axis=1)

In [48]:
label_count = df.midas_entity.value_counts().rename_axis('label').reset_index(name='counts')
label_count.head()

,label,counts
0,statement product,1187
1,statement location,548
2,opinion product,388
3,statement business,360
4,yes_no_question product,322


In [49]:
rare_midas_entity = label_count[label_count.counts <= 11].label.tolist()

In [50]:
df_common_midas_entity = df[~df.midas_entity.isin(rare_midas_entity)]
common_sampled_by_midas_and_entity = df_common_midas_entity.groupby(
    'midas_entity', group_keys=False).apply(lambda x: x.sample(10))

In [51]:
common_sampled_by_midas_and_entity[['midas_entity', 'text']].to_csv('data/daily_midas_entity_sample.tsv', sep='\t', index=False)

In [52]:
df_rare_midas_entity = df[df.midas_entity.isin(rare_midas_entity)]
rare_sampled_by_midas_and_entity = df_rare_midas_entity.groupby('midas_entity', group_keys=False).apply(lambda x: x.sample(1))

In [53]:
rare_sampled_by_midas_and_entity[['midas_entity', 'text']].to_csv('data/daily_midas_entity_rare_sampled.tsv', sep='\t', index=False)

In [54]:
df[['midas_entity', 'text']].groupby(
    'midas_entity', group_keys=False).apply(lambda x: x.sample(frac=1.0)).to_csv(
    'data/daily_all_target_sentences.tsv', sep='\t', index=False)

## Topical

In [92]:
Counter([len(sample['predict']['entities']) for sample in topical])

Counter({3: 5415,
         0: 119214,
         8: 45,
         1: 22442,
         2: 11780,
         4: 2237,
         6: 346,
         5: 867,
         7: 118,
         9: 22,
         11: 2,
         10: 5,
         19: 1})

In [114]:
target_sents = [sample['predict'] for sample in topical]
target_sents = [sample for sample in target_sents if len(sample['entities']) == 1]
for sample in target_sents:
    sample['entities'] = sample['entities'][0]
    
len(target_sents)

22442

In [115]:
df = pd.json_normalize(target_sents)

In [116]:
df[df.midas == "yes_no_question"].head()

,text,midas,entities.label,entities.offsets,entities.text
0,did you know google had hundreds of live goats to cut the LOCATION in the past?,yes_no_question,location,"[58, 63]",grass
2,did you know that a seahorse is the only fish to have a PRODUCT?,yes_no_question,product,"[56, 60]",neck
9,do you know how PRODUCT calculates traffic?,yes_no_question,product,"[16, 27]",google maps
26,do you know how many idnividual animals are believed to be on LOCATION?,yes_no_question,location,"[62, 67]",earth
37,was it a smuggling WORK_OF_ART?,yes_no_question,work_of_art,"[19, 24]",thing


In [117]:
df[['midas']].value_counts()

midas                
opinion                  10113
statement                8271 
yes_no_question          1546 
comment                  1109 
pos_answer               403  
command                  303  
open_question_factual    285  
open_question_opinion    217  
neg_answer               105  
complaint                43   
appreciation             21   
dev_command              13   
other_answers            13   
dtype: int64

In [118]:
df[['entities.label']].value_counts()

entities.label           
product                      4297
location                     2395
film                         2229
work_of_art                  1983
person                       1448
org                          1433
business                     1279
occupation                   1083
type_of_sport                903 
literary_work                804 
event                        511 
song                         472 
academic_discipline          471 
music_genre                  349 
food                         334 
norp                         331 
law                          325 
sport_team                   303 
animal                       283 
fac                          256 
nation                       216 
language                     186 
political_party              124 
sports_event                 107 
sports_season                88  
championship                 76  
sports_venue                 72  
sports_league                32  
chemical_element      

In [129]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

df[df['entities.label'] =='chemical_element']['entities.text'].value_counts()
# df[(df['entities.label'] =='work_of_art')]
# df[(df['entities.label'] =='work_of_art') & (df['entities.text'] =='bye')]

C:\Users\denis\AppData\Local\Temp/ipykernel_10212/3269599403.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


vitamin     9
helium      4
electron    2
types       2
bloods      1
fusion      1
proteins    1
dopamine    1
gold        1
enzyme      1
energy      1
hydrogen    1
gas         1
isotopes    1
Name: entities.text, dtype: int64

In [130]:
df.head()

,text,midas,entities.label,entities.offsets,entities.text
0,did you know google had hundreds of live goats to cut the LOCATION in the past?,yes_no_question,location,"[58, 63]",grass
1,i like PRODUCT.,opinion,product,"[7, 20]",google chrome
2,did you know that a seahorse is the only fish to have a PRODUCT?,yes_no_question,product,"[56, 60]",neck
3,yes that is a lot of PRODUCT and also a lot of money,statement,product,"[21, 26]",shoes
4,SONG riddance.,pos_answer,song,"[0, 13]",yes like good


In [132]:
rare_midas = []
rare_entity = []
df_common_midas = df[~df.midas.isin(rare_midas)]
df_rare_midas = df[df.midas.isin(rare_midas)]
df_common_entity = df[~df['entities.label'].isin(rare_entity)]

In [133]:
common_sampled_by_midas = df_common_midas.groupby('midas', group_keys=False).apply(lambda x: x.sample(10))
common_sampled_by_entity = df_common_entity.groupby('entities.label', group_keys=False).apply(lambda x: x.sample(10))
rare_sampled_by_midas = df_rare_midas.groupby('midas', group_keys=False).apply(lambda x: x.sample(1))

In [134]:
pd.concat([common_sampled_by_midas[['midas', 'text']], 
           rare_sampled_by_midas[['midas', 'text']]]).to_csv('data/topical_midas_sample.tsv', sep='\t', index=False)

In [136]:
common_sampled_by_entity[['entities.label', 'text']].to_csv('data/topical_entity_sample.tsv', sep='\t', index=False)

In [137]:
df['midas_entity'] = df[['midas', 'entities.label']].agg(' '.join, axis=1)

In [143]:
label_count = df.midas_entity.value_counts().rename_axis('label').reset_index(name='counts')
label_count.head()

,label,counts
0,opinion product,1811
1,statement product,1736
2,statement location,1128
3,opinion work_of_art,1072
4,opinion location,966


In [144]:
rare_midas_entity = label_count[label_count.counts <= 11].label.tolist()

In [145]:
df_common_midas_entity = df[~df.midas_entity.isin(rare_midas_entity)]
common_sampled_by_midas_and_entity = df_common_midas_entity.groupby(
    'midas_entity', group_keys=False).apply(lambda x: x.sample(10))

In [146]:
common_sampled_by_midas_and_entity[['midas_entity', 'text']].to_csv('data/topical_midas_entity_sample.tsv', sep='\t', index=False)

In [147]:
df_rare_midas_entity = df[df.midas_entity.isin(rare_midas_entity)]
rare_sampled_by_midas_and_entity = df_rare_midas_entity.groupby('midas_entity', group_keys=False).apply(lambda x: x.sample(1))

In [148]:
rare_sampled_by_midas_and_entity[['midas_entity', 'text']].to_csv('data/topical_midas_entity_rare_sampled.tsv', sep='\t', index=False)

In [149]:
df[['midas_entity', 'text']].groupby(
    'midas_entity', group_keys=False).apply(lambda x: x.sample(frac=1.0)).to_csv(
    'data/topical_all_target_sentences.tsv', sep='\t', index=False)